# Race analysis

### Kyle Willett ([@willettk](https://github.com/willettk))

Some time-wasting ways of filtering and sorting my personal data from running races. 

In [ ]:
# Get them packages

import datetime
import re
from collections import Counter
from operator import itemgetter

import pandas as pd
import numpy as np

from IPython.display import display

In [ ]:
# Running data can be either from live website or local file

getLocalData = True
localFile = "/Users/willettk/willettk.github.io/content/racelist.html"
url = "http://willettk.github.io/racelist.html"

dataIO = localFile if getLocalData else url
try:
    # Select only data for solo running races; relays, bike races, triathlons are all in separate tables
    attrs = {'id': 'runtable'}
    races = pd.read_html(dataIO, attrs = attrs)
except ValueError:
    print("Could not find table matching attributes {} on page.".format(attrs))

In [ ]:
# Pre-process data as pandas DataFrame

run = races[0]

# Rename columns for some easier typing
rc = run.columns
run.rename(columns={rc[0]:"date",
                    rc[1]:"race",
                    rc[2]:"location",
                    rc[3]:"d_km",
                    rc[4]:"d_mi",
                    rc[5]:"pace",
                    rc[6]:"time",
                    rc[7]:"place_overall",
                    rc[8]:"finishers_overall",
                    rc[9]:"place_division",
                    rc[10]:"finishers_division",
                    rc[11]:"division"
                   },inplace=True)

run_columns = ['date','race','location','time','pace',
               'place_overall','finishers_overall',
               'place_division','finishers_division']
    
# Convert to int where possible
for col in ('place_overall','finishers_overall','place_division','finishers_division'):
    run[col] = run[col].fillna(-1).astype(int)

# Ditch races where finishing data is probably inaccurate, based on lack of pace
run = run[[False if type(x) == float and np.isnan(x) else True for x in run.pace]]

In [ ]:
def parse_races(dt):

    # Transform date, time, and pace into numerical objects
    
    try:
        dt['date'] = pd.to_datetime(dt.date, format='%m/%d/%y')
    except AttributeError:
        print(dt)
    dt['pace'] = [datetime.timedelta(minutes=float(x.split()[0].split(':')[0]),
                                         seconds=float(x.split()[0].split(':')[1])) for x in dt.pace]
    try:
        dt['time'] = [datetime.timedelta(hours=float(x.split(':')[0]),
                                         minutes=float(x.split(':')[1]),
                                         seconds=float(x.split(':')[2])) for x in dt.time]
    except IndexError:
        dt['time'] = [datetime.timedelta(minutes=float(x.split(':')[0]),
                                     seconds=float(x.split(':')[1])) for x in dt.time]


    # Restrict to races with data on overall and division placing

    dtf = dt[np.isfinite(dt['finishers_overall']) & np.isfinite(dt['finishers_division'])].copy()
    for c in dtf.columns[-4:]:
        dtf[c] = dtf[c].astype(int)
    
    return dt,dtf

In [ ]:
def filter_races(distance=None):

    # Filter for races at a given distance (rounded to nearest tenth of a mile)
    
    if distance != None:
        dt = run.copy()[run['d_mi'].round(1) == distance][run_columns]
    else:
        dt = run.copy()[run_columns]

    dt,dtf = parse_races(dt)
    
    return dt,dtf

In [ ]:
def distinct_places(df):
    # Find distinct states/polities for a set of races
    return Counter([l.split(",")[-1].strip() for l in df.location])

In [ ]:
def more_than_once(df):
    # Find races run more than once
    c = Counter(df.race)
    races,count = [],[]
    for r in c:
        if c[r] > 1:
            races.append(r)
            count.append(c[r])
    
    return pd.DataFrame({'race':races},index=count).sort_index(ascending=False)

In [ ]:
def time_formatting(t,verbose=False):
    # Output times in something sensibly human-readable
    if t.seconds > 3600:
        if verbose:
            print("Formatting as HH:MM:SS")
        timestr = "{:.0f}:{:02.0f}:{:02.0f}".format(int(t.seconds / 3600), int((t.seconds % 3600)/60), t.seconds % 60 )
    elif t.seconds > 60:
        if verbose:
            print("Formatting as MM:SS")
        timestr = "{:.0f}:{:02.0f}".format(int(t.seconds / 60), t.seconds % 60 )
    else:
        if verbose:
            print("Formatting as SS")
        timestr = "{:.0f}".format(t.seconds)

    return timestr

In [ ]:
def personal_best(df):
    # Return personal best time at a given distance
    best = df.sort_values("time").reset_index().loc[0]
    timestr = time_formatting(best.time)

    race = best.race
    year = best.date.year
    d = {'time':timestr,'race':race,'year':year}
    
    return d

In [ ]:
def plural_stem(s):
    return "" if s == 1 else "s"

In [ ]:
def summarize(distance=None):
    
    # Print out everything prettily
    dt,dtf = filter_races(distance) 
    n = len(dt)
    
    placeListRaw = distinct_places(dt).items()
    placeList = [(x[0],x[1],len(x[0])) for x in placeListRaw]
    for key,reverseOrder in zip((0,2,1),(False,False,True)):
        placeList.sort(key=itemgetter(key),reverse=reverseOrder)

    polities = re.sub("['\[\]]","",str(["{} ({})".format(x[0], x[1]) for x in placeList]))
    
    # Races split by location
    
    if distance != None:
        print("\nI've run {} race{} of {} mile{}.\n".format(n, plural_stem(n), distance, plural_stem(distance)))
        
        if n > 0:
            print("Personal best: {time}, set at {race} in {year}.\n".format(**personal_best(dt)))

            print("I've run {} mile{} in {}.\n".format(distance, plural_stem(distance), polities))
        
        distanceStr = '{} mile-'.format(distance)
        
    else:
        print("I've run races in {}.\n".format(polities))
        distanceStr = ''
        print("\nI've run {} total races.".format(len(dt)))    
    
    # Races run more than once

    mo = more_than_once(dt)
    if len(mo) > 0:
        if distance != None:
            print("\nRaces of {} mile{} that I've run more than once:".format(distance, plural_stem(distance)))
        else:
            print("\nRaces that I've run more than once:")
        display(mo)
    else:
        print("\nI've never run the same {}race more than once.".format(distanceStr))


### For a given distance, summarize:

* number of races
* locations
* personal best
* races run more than once

In [ ]:
# Only summarize the N most common distances
nd = 6

mcd = [round(float(x[0]),1) for x in Counter(run['d_mi']).most_common(nd)]
mcd.sort()
for d in mcd:
    summarize(d)

In [ ]:
# Summarize races over all distances

summarize()

### How has my personal best for each distance progressed?

In [ ]:
def personal_best_progression(distance=13.1):
    
    # In ascending chronological order for a given distance,
    # print out all races which set or equalled a previous personal best time.
    
    dt,dtf = filter_races(distance) 
    n = len(dt)
    if n > 0:
        firstrace = dt.iloc[0]
        best = firstrace.time
        bestyear = firstrace.date.year
        timestr = time_formatting(firstrace.time)
        print("Personal best progression of {} miles ({} race{}):\n".format(distance,n,plural_stem(n)))
        print("\tFirst run {}: {} at {}.".format(firstrace.date.year,timestr,firstrace.race))
        for i in range(n-1):
            row = dt.iloc[i+1]
            if row.time <= best:
                timestr_new = time_formatting(row.time)
                print("\tNew PB in {}: {} at {}.".format(row.date.year,timestr_new,row.race,))
                best = row.time
    else:
        print("No races found for distance of {} miles.".format(distance))

    return None

In [ ]:
# Example of progression of personal bests

d = 3.1
personal_best_progression(d)

### How have I done, year over year, in setting personal bests?

In [ ]:
# Only consider PBs at the most common/iconic distances. 
distances = {
    1:"1 mile",
    3.1:"5 km",
    6.2:"10 km",
    13.1:"half marathon",
    26.2:"marathon",
}

distances_rev = {v:k for k,v in distances.items()}

In [ ]:
# Find range of years of active running
pb = {}
start_year = pd.to_datetime(run.iloc[0].date, format='%m/%d/%y').year
this_year = datetime.datetime.now().year
for year in range(start_year,this_year+1):
    pb[year] = []

# Append if a PB is set for any of the selected distances
for distance in distances.keys():
    dt,dtf = filter_races(distance) 
    n = len(dt)
    if n > 0:
        firstrace = dt.iloc[0]
        best = firstrace.time
        pb[firstrace.date.year].append(distances[distance])
        for i in range(n-1):
            row = dt.iloc[i+1]
            if row.time <= best:
                pb[row.date.year].append(distances[distance])
                best = row.time

# Print list of results for each year
years = sorted(list(pb.keys()))
for year in years:
    sorted_pbs = sorted(list(set(pb[year])),key = lambda x: distances_rev[x])
    print(year, sorted_pbs if len(pb[year]) > 0 else None)

2016 was a really good year for me - PRs at four distances, from 1 mile up to the marathon. And I've been lucky to be consistently improving, even well into my 30s; except for my break from running in 2005 and 2006, I've set a PR at one of the standard distances every single year. 

#### Number of races per year

Plot the total number of races per year and label the maximum.

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
def label_max(histobj,ax,title):
    counts,yearbins,desc = histobj
    argmax = counts.argmax()
    ax.text(yearbins[argmax],counts[argmax],"{:.0f}: {:.0f} {}".format(yearbins[argmax]+eps,counts[argmax],title)
         ,va='bottom'
         ,color='red')

In [ ]:
# Number of races per year
years = [x.year for x in pd.to_datetime(run.date, format='%m/%d/%y')]

# Number of total miles run each year in races
z = [(d.year,int(m)) for d,m in zip(pd.to_datetime(run.date, format='%m/%d/%y'),run.d_mi)]
l = [[a,]*b for a,b in z]
flat_list = [item for sublist in l for item in sublist]

# Number of total miles run each year in races
z = [(d.year,int(m)) for d,m in zip(pd.to_datetime(run.date, format='%m/%d/%y'),run.d_mi)]
l = [[a,]*b for a,b in z]
flat_list = [item for sublist in l for item in sublist]

fig,axarr = plt.subplots(1,2,figsize=(16,7))
eps = 0.5
bins = np.arange(min(years)-eps,max(years)+eps+1,1)

for ax,data,label in zip(axarr,(years,flat_list),('races','miles')):
    histobj = ax.hist(data,bins=bins)
    ax.set_xlabel("Year",fontsize=14)
    ax.set_ylabel(label.capitalize(),fontsize=14)
    label_max(histobj,ax,label);

### Total race miles

In [ ]:
print("I've run {:.0f} total miles in {:d} races.".format(run['d_mi'].sum(),len(run)))

In [ ]:
d = 6.2
run[run['d_mi'] == d].sort_values(["d_km","time","date"],axis=0,ascending=[1,1,0])